# Computational physics 2, Exercise 1

## Random Number Generators
The traditional random number generator (RNG) in computer science generates a sequence of the form
Xn+1 = (aXn + c) mod k. (1)

(a) Verify the point made in the lecture that such pseudo-random numbers are highly correlated.

(b) Try k = 256 and k = 1024 and choose good parameters for a and c. Can you explain the behavior?

(c) The performance of a bad RNG can be vastly improved by coupling two bad RNGs. We want to
study this possibility. Run two differently seeded versions (also try to use different a, c, and k) of
the above bad RNG. A random number from RNG 1 is only taken if RNG 2 produces a number
that is a multiple of either 2, 3, 5, 7, or 13. Study the correlation of this new RNG

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
print("Imported numpy and matplotlib")

In [ ]:
from typing import Iterator
def linear_congruential_rng(x0: int, a: int, c: int, k: int) -> Iterator[int]:
    x = x0
    while True:
        x = (a*x + c) % k
        yield x

In [ ]:
rng = linear_congruential_rng(10, 27, 13, 256)
random_pairs = [(next(rng), next(rng)) for _ in range(512)]
random_x = [p[0] for p in random_pairs]
random_y = [p[1] for p in random_pairs]

plt.figure()
plt.title("Task 1 a)")
plt.scatter(random_x, random_y)

In [ ]:
rng_256 = linear_congruential_rng(0, 237, 2, 256)
random_pairs_256 = [(next(rng_256), next(rng_256)) for _ in range(2048)]
random_x_256 = [p[0] for p in random_pairs_256]
random_y_256 = [p[1] for p in random_pairs_256]

rng_1024 = linear_congruential_rng(0, 237, 2, 1024)
random_pairs_1024 = [(next(rng_1024), next(rng_1024)) for _ in range(2048)]
random_x_1024 = [p[0] for p in random_pairs_1024]
random_y_1024 = [p[1] for p in random_pairs_1024]

plt.figure()
plt.suptitle("Task 1 b)")
plt.subplot(1,2,1)
plt.scatter(random_x_256, random_y_256)
plt.subplot(1,2,2)
plt.scatter(random_x_1024, random_y_1024)


In [ ]:
def coupled_linear_congruential_rng(x0: int, a1: int, c1: int, k1: int, a2: int, c2: int, k2: int) -> Iterator[int]:
    x1 = x0
    x2 = x0
    rng1 = linear_congruential_rng(x1, a1, c1, k1)
    rng2 = linear_congruential_rng(x2, a2, c2, k2)
    accepted_divisors = [2, 3, 5, 7, 13]
    while True:
        x2 = next(rng2)
        while all(x2 // i for i in accepted_divisors):
            x1 = next(rng1)
            x2 = next(rng2)
        yield x1
    

In [ ]:
rng = coupled_linear_congruential_rng(0, 237, 1, 256, 237, 1, 1024)
random_pairs = [(next(rng), next(rng)) for _ in range(1000)]
random_x = [p[0] for p in random_pairs]
random_y = [p[1] for p in random_pairs]

plt.figure()
plt.title("Task 1 c)")
plt.scatter(random_x, random_y)

## Numerical integration

$$ \int_0^\infty e−x dx $$
$$ \int_0^\pi \sin(x) dx $$
$$ \int_0^1 \frac{\ln(\cos(x))}{x} dx $$

(a) In the last term, we discussed numerical integration of a known function. This exercise is meant to
warm up your skills on this topic of Computational Physics I. Use the Trapezoidal rule and Simpson’s
rule to compute the integrals given above. Compare the relative accuracy of the result to the
number of function evaluations needed. The numerical value of Eq. (4) is −0.27568727380043716 · · · .
Visualize the result in a log-log plot and determine the rate of convergence for each method. What
rate would you expect? You can find a code skeleton in the code section of the homepage, which
includes a plotting script.

(b) Implement Monte Carlo integration.
- Implement two different versions of Monte Carlo integration (by-rejection, by-mean) and solve
the above integrals. Use a linear transformation from the interval of your random numbers
[0, 1] to the integration boundaries [a, b].
- Try to use estimates for the extremal values of the integrand for the by-rejection integration
that are two orders of magnitude too large. Can you explain the behavior of the algorithm
performance?
- Now, try to use importance sampling for those integrals where it is sensible and potentially
improves the integration. Find a suitable probability density function close/similar to the
integrand to draw from; do not use the integrand itself.
- Which of all your algorithms performs best? For which problems should you use Monte Carlo
integration?

In [ ]:
from scipy.integrate import simpson as scipy_simpson
from typing import Callable

def trapezoidal(f: Callable[[np.ndarray], np.ndarray], a: float, b: float, N: int) -> float:
    x = np.linspace(a, b, N)
    y = f(x)
    return np.trapz(y, x)

def simpson(f: Callable[[np.ndarray], np.ndarray], a: float, b: float, N: int) -> float:
    x = np.linspace(a, b, N)
    y = f(x)
    return scipy_simpson(y, x)


In [ ]:
def plot_integral_comparison(f: Callable[[np.ndarray], np.ndarray], a: float, 
                             b: float, integral_value: float, title: str):
    N = np.logspace(2, 6, 100, dtype=np.int64) // 2 + 1
    trapz = [abs(trapezoidal(f, a, b, n) - integral_value) for n in N]
    simps = [abs(simpson(f, a, b, n) - integral_value) for n in N]

    plt.figure()
    plt.title(title)
    plt.loglog(N, trapz, label="Trapezoidal")
    plt.loglog(N, simps, label="Simpson")
    plt.legend()
    plt.xlabel("N")

In [ ]:
def f(x):
    return np.exp(-x)
integral_value = 1
plot_integral_comparison(f, 0, 100, integral_value, "$e^{-x}$")


In [ ]:
plot_integral_comparison(np.sin, 0, np.pi, 2, "$\\sin x$")


In [ ]:
def f(x):
    y = np.log(np.cos(x)) / x
    y[x == 0] = 0
    return y
plot_integral_comparison(f, 0, 1, -0.27568727380043716, "$\\frac{\\ln\\cos x}{x}$")

test two